1. Objectif du nettoyage

L'objectif de cette étape est des transformer les données brutes en données exploitables pour l'analyse, sans en modifier le sens ou la granularité.

Le nettoyage vise à :
- standardiser les noms de colonnes pour faciliter leur manipulation en Python/SQL
- identifier et formater la dimension temporelle
- filtrer un périmètre cohérent (géographique et périodique)
- sélectionner les variables utiles pour l'analyse
- préparer une table propre et stable pour les étapes suivantes

2. Chargement des données

In [5]:
import pandas as pd
df_raw_dares = pd.read_csv("../data/raw/france_travail/dares_defm_stock_france_brut_trim.csv", sep =";")

In [6]:
df_raw_dares.head()

,Date,Champ,Type de données,Catégorie,Sexe,Tranche d'âge,Tranche d'heures travaillées,Ancienneté,Formation,Formation (ancienne mesure),Métier recherché,Indemnisabilité,Qualification,Nombre de demandeurs d'emploi
0,1996-T1,France métropolitaine,Brutes,C,Total,Total,Total,Total,Total,Total,Total,Total,Total,324100
1,1996-T1,France métropolitaine,Brutes,C,Hommes,Total,Total,Total,Total,Total,Total,Total,Total,148800
2,1996-T1,France métropolitaine,Brutes,D,Total,Total,Total,Total,Total,Total,Total,Total,Total,181500
3,1996-T1,France métropolitaine,Brutes,E,Total,Total,Total,Total,Total,Total,Total,Total,Total,202200
4,1996-T1,France métropolitaine,Brutes,A,Hommes,Moins de 25 ans,Total,Total,Total,Total,Total,Total,Total,315600


In [10]:
df_dares = df_raw_dares.copy()

In [35]:
df_raw_insee20 = pd.read_csv("../data/raw/insee/FD_csv_EEC20.csv", sep=";")
df_raw_insee21 = pd.read_csv("../data/raw/insee/FD_EEC_2021.csv", sep=";")
df_raw_insee22 = pd.read_csv("../data/raw/insee/FD_EEC_2022.csv", sep=";")
df_raw_insee23 = pd.read_csv("../data/raw/insee/FD_csv_EEC23.csv", sep=";")

In [36]:
df_raw_insee = pd.concat([df_raw_insee20, df_raw_insee21, df_raw_insee22, df_raw_insee23])

In [38]:
df_insee = df_raw_insee.copy()

In [39]:
df_insee.columns

Index(['ANNEE', 'TRIM', 'CATAU2010R', 'METRODOM', 'TYPMEN7', 'AGE3', 'AGE5',
       'COURED', 'ENFRED', 'NFRRED',
       ...
       'APCS2', 'CHAMP_M_15_89', 'ESEG_1', 'ESEG_2', 'NATIO', 'PCSL', 'PCSP',
       'SO', 'STAT_CS', 'STAT_CSPRL'],
      dtype='object', length=177)

In [42]:
df_insee["periode"] = (
    pd.PeriodIndex.from_fields(
        year=df_insee["ANNEE"].astype(int),
        quarter=df_insee["TRIM"].astype(int),
        freq="Q"
    )
    .to_timestamp(how="start")
)

In [47]:
df_insee[["ANNEE", "TRIM", "periode"]].head()

,ANNEE,TRIM,periode
0,2020,4,2020-10-01
1,2020,1,2020-01-01
2,2020,1,2020-01-01
3,2020,2,2020-04-01
4,2020,2,2020-04-01


In [49]:
df_insee = df_insee[(df_insee["ANNEE"] >= 2020) & (df_insee["ANNEE"] <= 2023)]

In [51]:
keep_cols = [
    "ANNEE", "TRIM", "periode",
    "METRODOM",
    "NATIO",
    "SEXE",
    "AGE3", "AGE5",
    "PCSL", "PCSP", "STAT_CS", "STAT_CSPRL",
    "SO"
]

keep_cols = [c for c in keep_cols if c in df_insee.columns]
df_insee = df_insee[keep_cols]
df_insee.head()

,ANNEE,TRIM,periode,METRODOM,NATIO,SEXE,AGE3,AGE5,PCSL,PCSP,STAT_CS,STAT_CSPRL,SO
0,2020,4,2020-10-01,1,NaN,2,50.0,60.0,NaN,NaN,NaN,NaN,NaN
1,2020,1,2020-01-01,1,NaN,1,50.0,60.0,NaN,NaN,NaN,NaN,NaN
2,2020,1,2020-01-01,1,NaN,2,50.0,60.0,NaN,NaN,NaN,NaN,NaN
3,2020,2,2020-04-01,1,NaN,1,50.0,60.0,NaN,NaN,NaN,NaN,NaN
4,2020,2,2020-04-01,1,NaN,2,50.0,60.0,NaN,NaN,NaN,NaN,NaN


In [52]:
df_insee.shape

(1360290, 13)

In [53]:
df_insee.isna().mean().sort_values(ascending=False).head(10)

PCSP          0.779976
AGE3          0.765199
AGE5          0.765199
STAT_CSPRL    0.748908
PCSL          0.748907
STAT_CS       0.748313
NATIO         0.743713
SO            0.743713
ANNEE         0.000000
TRIM          0.000000
dtype: float64

In [54]:
df_insee.duplicated().sum()

np.int64(1181066)

In [55]:
[c for c in df_raw_insee.columns if "SEXE" in c]

['SEXE']

In [56]:
df_insee[["ANNEE","TRIM"]].drop_duplicates().sort_values(["ANNEE","TRIM"]).head(20)

,ANNEE,TRIM
1,2020,1
3,2020,2
5,2020,3
0,2020,4
0,2021,1
86384,2021,2
172268,2021,3
256830,2021,4
0,2022,1
88589,2022,2


In [59]:
df_raw_varmod20 = pd.read_csv("../data/raw/insee/varmod/Varmod_EEC_2020.csv", sep=";")
df_raw_varmod21 = pd.read_csv("../data/raw/insee/varmod/Varmod_EEC_2021.csv", sep=";")
df_raw_varmod22 = pd.read_csv("../data/raw/insee/varmod/Varmod_EEC_2022.csv", sep=";")
df_raw_varmod23 = pd.read_csv("../data/raw/insee/varmod/Varmod_EEC_2023.csv", sep=";")

In [63]:
df_raw_varmod = pd.concat([df_raw_varmod20, df_raw_varmod21, df_raw_varmod22, df_raw_varmod23])

In [64]:
df_varmod = df_raw_varmod.copy()

In [68]:
df_varmod.head(3)

,COD_VAR,LIB_VAR,COD_MOD,LIB_MOD,TYPE_VAR,LONG_VAR
0,ANNEE,Année de référence de l'enquête,2020,Année de référence sur 4 positions,CHAR,4.0
1,TRIM,Trimestre de référence de l'enquête,1,1er trimestre de l'année,CHAR,1.0
2,TRIM,Trimestre de référence de l'enquête,2,2e trimestre de l'année,CHAR,1.0


In [79]:
keywords = [
    "ancienneté", "emploi", "chômage", "chomage", "activité", "actif", "statut", "profession", "categorie", "catégorie", "pcs", "sexe", "âge", "age"
]
mask = df_varmod["LIB_VAR"].str.lower().str.contains("|".join(keywords), na=False)
vars_interessantes = df_varmod.loc[mask, ["COD_VAR", "LIB_VAR"]].drop_duplicates()

vars_interessantes.sort_values("COD_VAR").head(30)

,COD_VAR,LIB_VAR
766,AAC,Exercice d'une activité professionnelle réguli...
4,ACL_EMPLOI,Classe d'emploi (emploi actuel ou dernier emploi)
4,ACTEU,Statut d'activité au sens du Bureau Internatio...
38,ACTEU,Statut d'activité au sens du Bureau Internatio...
42,ACTEU6,Statut d'activité au sens du Bureau Internatio...
48,ACTIF,Actif au sens du BIT
51,ACTOP,Actif occupé au sens du Bureau International d...
18,AGE3,Âge au dernier jour de la semaine de référence...
22,AGE5,Âge au dernier jour de la semaine de référence...
35,AISCO2,"Code ISCO (dernier emploi, niveau 2)"


In [80]:
candidates = vars_interessantes["COD_VAR"].unique().tolist()
existing = [c for c in candidates if c in df_insee.columns]

len(candidates), len(existing)

(139, 8)

In [81]:
vars_existantes = vars_interessantes[vars_interessantes["COD_VAR"].isin(existing)]
vars_existantes.head(40)

,COD_VAR,LIB_VAR
18,AGE3,Âge au dernier jour de la semaine de référence...
22,AGE5,Âge au dernier jour de la semaine de référence...
36,SEXE,Sexe
1169,PCSL,PCS Logement calculée à partir du statut au se...
1187,PCSP,PCS Ménage des parents
1295,SO,Statut d'occupation du ménage dans le logement
1333,STAT_CS,Statut (au sens spontané) selon la catégorie s...
1442,STAT_CSPRL,Statut (au sens spontané) selon la catégorie s...


In [82]:
selected_vars = [
    "ANNEE", "TRIM", "periode",
    "AGE5", "SEXE",
    "STAT_CS", "STAT_CSPRL",
    "PCSL", "PCSP"
]

selected_vars = [c for c in selected_vars if c in df_insee.columns]

df_insee_clean = df_insee[selected_vars].copy()
df_insee_clean.shape, df_insee_clean.columns

((1360290, 9),
 Index(['ANNEE', 'TRIM', 'periode', 'AGE5', 'SEXE', 'STAT_CS', 'STAT_CSPRL',
        'PCSL', 'PCSP'],
       dtype='object'))

In [83]:
df_insee_clean

,ANNEE,TRIM,periode,AGE5,SEXE,STAT_CS,STAT_CSPRL,PCSL,PCSP
0,2020,4,2020-10-01,60.0,2,NaN,NaN,NaN,NaN
1,2020,1,2020-01-01,60.0,1,NaN,NaN,NaN,NaN
2,2020,1,2020-01-01,60.0,2,NaN,NaN,NaN,NaN
3,2020,2,2020-04-01,60.0,1,NaN,NaN,NaN,NaN
4,2020,2,2020-04-01,60.0,2,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
348619,2023,4,2023-10-01,NaN,1,365.0,365.0,VI-B,NaN
348620,2023,4,2023-10-01,NaN,2,352.0,352.0,VI-A,VII-A
348621,2023,4,2023-10-01,NaN,2,356.0,356.0,V-A,VII-A
348622,2023,4,2023-10-01,NaN,1,367.0,356.0,V-A,VI-B


In [84]:
df_varmod[df_varmod["COD_VAR"].isin(["STAT_CS","STAT_CSPRL","AGE5","SEXE","PCSL","PCSP"])][
    ["COD_VAR","COD_MOD","LIB_MOD"]
].drop_duplicates().sort_values(["COD_VAR","COD_MOD"]).head(60)

,COD_VAR,COD_MOD,LIB_MOD
22,AGE5,0,De 0 à 14 ans
23,AGE5,15,De 15 à 29 ans
24,AGE5,30,De 30 à 39 ans
25,AGE5,40,De 40 à 49 ans
26,AGE5,50,De 50 à 59 ans
27,AGE5,60,60 ans ou plus
1185,PCSL,999,Non codés
1169,PCSL,I-A,Cadre avec cadre
1170,PCSL,I-B,Cadre avec profession intermédiaire
1171,PCSL,II-A,Cadre avec employé ou ouvrier


In [85]:
df_insee_clean.shape

(1360290, 9)

In [86]:
final_vars = [
    "ANNEE", "TRIM", "periode",
    "AGE5", "SEXE",
    "STAT_CS", "STAT_CSPRL"
]

df_insee_clean = df_insee[final_vars].copy()
df_insee_clean.shape, df_insee_clean.columns

((1360290, 7),
 Index(['ANNEE', 'TRIM', 'periode', 'AGE5', 'SEXE', 'STAT_CS', 'STAT_CSPRL'], dtype='object'))

In [87]:
from pathlib import Path
Path("../data/processed").mkdir(parents=True, exist_ok=True)

df_insee_clean.to_parquet(
    "../data/processed/insee_eec_clean.parquet",
    index=False
)

3. Standardisation des colonnes

4. Gestion des dates

5. Filtrage du périmètre

6. Table finale